In [18]:
def create_Geodataframe(txt_sat_list,max_sat_num,time_list,user) :
    
    
    count_sat = 0
    
    # create empty geodataframe
    # ==============================================
    gdf_4326 = gpd.GeoDataFrame(columns=['SAT_name','SAT_ID','TIME','SAT_EL','SAT_AZ','SAT_lat','SAT_lon','SAT_h','geometry','State'],crs='4326')
    gdf_4326['geometry'] = None
    
    
    
    # use norad id to get satellite model until max satllite orbit number
    # ==============================================
    for norad_id in range(45184,45104,-1):
         
        if( count_sat >= max_sat_num ):
            break
        
        satellite  = getInfo_Satellite_Model(norad_id) # get satellite from norad id
        
        if (satellite.name == 'No GP data found') :            # check satellite found or not
            print('No GP data found')
            continue
            
        else :    
        
            file = open(txt_sat_list,"a")                  # write satellite id to satellite list
            file.write(str(norad_id)+'\n')
            file.close()
            count_sat = count_sat + 1 
            
            link_info = getInfo_Satellite_Position (satellite,time_list,user) 
                                                            # satellite position 
                                                            # using a time list 
                                                            # and save link info to class
            gdf_4326 = link_Info_Write_To_Gdf(gdf_4326,link_info,norad_id)
                    
    return gdf_4326

In [17]:
def getInfo_Satellite_Model(Norad_id) :
    
    time.sleep(3)
    
    try : 

        # load txt
        #===============================================================
        # url = 'https://celestrak.com/satcat/tle.php?CATNR={}'.format(Norad_id)
        # url 格式已被更改
        url = 'https://celestrak.org/NORAD/elements/gp.php?CATNR={}'.format(Norad_id)
        filename = 'data/SatelliteList_Celestrak/tle-CATNR-{}.txt'.format(Norad_id)
        satellite_log = load.tle_file(url, filename=filename)

        # open file
        # =============================================================
        fp = open('data/SatelliteList_Celestrak/tle-CATNR-{}.txt'.format(Norad_id), "r")
        sat_name = fp.readline()
        line1 = fp.readline()
        line2 = fp.readline()

        # print(sat_name)
        # print(line1)
        # print(line2)
        
        fp.close()

        # creat satellite
        # =============================================================
        ts = load.timescale()
        satellite = EarthSatellite(line1, line2, sat_name, ts)

        # check epoch
        # =============================================================
        utc_now = datetime.datetime.utcnow() 
        t = ts.utc(utc_now.year, utc_now.month, utc_now.day, utc_now.hour, utc_now.minute, utc_now.second)


        days = t - satellite.epoch
        #print('=================================================')
        #print('Info from : getInfo_TLE_from_Skyfield')
        #print('CHECK : {:.3f} days away from epoch'.format(days))
        #print('TIME  : ' + utc_now.strftime('%Y%m%d %H%M%S'))

        if abs(days) > 14:

            # satellites = load.tle_file(stations_url, reload=True)
            print('reload satellite : ' + str(Norad_id))

            url = 'https://celestrak.org/NORAD/elements/gp.php?CATNR={}'.format(Norad_id)
            filename = 'data/SatelliteList_Celestrak/tle-CATNR-{}.txt'.format(Norad_id)
            satellite_log = load.tle_file(url, filename=filename)

            # open updated file
            fp = open('data/SatelliteList_Celestrak/tle-CATNR-{}.txt'.format(Norad_id), "r")
            sat_name = fp.readline()
            line1 = fp.readline()
            line2 = fp.readline()

            # print(sat_name)
            # print(line1)
            # print(line2)

            fp.close()

            # creat updated satellite
            # =============================================================
            ts = load.timescale()
            satellite = EarthSatellite(line1, line2, sat_name, ts)
            days = t - satellite.epoch

        return satellite
        
    except : 
        
        print('satellite from skyfield not found ')
    

In [15]:
def getInfo_Satellite_Position (satellite,time_list,user) :
    
    link_info = Link_Info ()
    
    ts = load.timescale()
    user_loc = wgs84.latlon(user.Latitude,user.Longitude,user.Height)
    link_info.SAT_name = satellite.name
    link_info.TimeStep = time_list.Step

    start = time_list.Start
    end   = time_list.End
    
    t0 = ts.utc(start.year, start.month, start.day)
    t1 = ts.utc(  end.year,   end.month,   end.day)
    
    time_buffer_start = None
    time_buffer_end   = None
    
    t, events = satellite.find_events(user_loc, t0, t1, altitude_degrees=0.0) 
                            # altitude_degrees means the threshold of 
                            # definition angle of set and rise
                    
                            # event :
                            # 0 : rise
                            # 1 : culminate
                            # 2 : set
    for ti, event in zip(t, events):
        name = ('rise above 0°', 'culminate', 'set below 0°')[event]
        
        if(event == 0) :    
            time_buffer_start = ti.utc_strftime('%Y%m%d%H%M%S')
        if(event == 2) :
            time_buffer_end   = ti.utc_strftime('%Y%m%d%H%M%S')
            
        if(time_buffer_start != None and time_buffer_end != None) :
            
            # if we have timestamp of SAT rise to SAT set, then ...
            
            dt_start = datetime.datetime.strptime(time_buffer_start,"%Y%m%d%H%M%S")
            dt_end   = datetime.datetime.strptime(time_buffer_end  ,"%Y%m%d%H%M%S")
            
            
            # write user location and satellite location to link info 
            # for every timestamp 
            # in the interval from SAT rise to SAT set 
            # ==================================================================================
            date = dt_start
            count = 1
            while( (dt_end-date).total_seconds() > 0 ) :
                
                link_info.Time.append(date)
                link_info.User_loc.append(user_loc)

                # SAT geocentric at time geo_t
                # ===============================================
                geo_t = ts.utc(date.year, date.month, date.day, date.hour, date.minute, date.second)
                geocentric = satellite.at(geo_t)

                # SAT lat,lon, and height
                # ===============================================
                lat, lon = wgs84.latlon_of(geocentric)
                height = wgs84.height_of(geocentric)
                
                link_info.SAT_Latitude.append(lat)
                link_info.SAT_Longitude.append(lon)
                link_info.SAT_Height.append(height)

                # set the elvation of ground at the same lat,lon
                # ===============================================
                elevation_m = 0 
                subpoint = wgs84.latlon(lat.degrees, lon.degrees, elevation_m)


                difference = satellite - user_loc # 衛星距離參考點相對差
                diff_centric = difference.at(geo_t)


                # 相對位置以alt,az 表示
                # ===============================================
                alt, az, distance = diff_centric.altaz()
                # if alt.degrees > 0:
                #     print('The SAT is above the horizon')

                # class 'skyfield.units.Angle'
                # ===============================================
                link_info.Diff_Altitude.append(alt)
                link_info.Diff_Azimuth.append(az)
                link_info.Diff_Distance.append(distance)
                # print('diff Altitude:', alt) # 方向角
                # print('diff Azimuth:', az)   # 方位角
                # print('diff Distance: {:.1f} km'.format(distance.km))

                # 相對位置以 ra,dec 表示
                # ===============================================
                ra, dec, distance = diff_centric.radec()  # ICRF ("J2000")
                link_info.Diff_RA.append(ra)
                link_info.Diff_De.append(dec)
                link_info.Diff_dis.append(distance)

                # print('diff of right ascension: ',ra)
                # print('diff of declination    : ',dec)

                # range rate for doppler shift
                # ===============================================
                pos = (satellite - user_loc).at(geo_t)
                _, _, the_range, _, _, range_rate = pos.frame_latlon_and_rates(user_loc)

                link_info.range.append(the_range)
                link_info.range_rate.append(range_rate)
                # print('range     :',array2string(the_range.km, precision=1), 'km')
                # print('range_rate:',array2string(range_rate.km_per_s, precision=2), 'km/s')

                date = date + datetime.timedelta(seconds=int(time_list.Step))
                # print('time_list.Step : ' + str(time_list.Step))
            
            # ==================================================================================
            
            clear_output(wait=True)
            print('create link info : ',satellite.name,flush=True)
            print('sat link size   : ',len(link_info.Time))
            
            # print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)
            time_buffer_start = None
            time_buffer_end   = None
        
        
    return link_info

In [16]:
def link_Info_Write_To_Gdf(gdf_4326,link_info,norad_id) :
    
    for i in range(0,len(link_info.Time)) :
        
        SAT_name = link_info.SAT_name
        SAT_ID   = norad_id
        TIME     = link_info.Time[i]
        #TIME     = datetime.datetime.strptime(link_info.Time[i],"%Y_%m%d_%H%M%S")
        #2022_0927_055636	
        
        SAT_EL   = link_info.Diff_Altitude[i].degrees
        SAT_AZ   = link_info.Diff_Azimuth[i].degrees
        SAT_lat  = link_info.SAT_Latitude[i].degrees
        SAT_lon  = link_info.SAT_Longitude[i].degrees
        SAT_h    = link_info.SAT_Height[i].km
        
        #geometry
        d_sat_lat = link_info.SAT_Latitude[i].degrees
        d_sat_lon = link_info.SAT_Longitude[i].degrees
        
        d_user_lat = link_info.User_loc[i].latitude.degrees
        d_user_lon = link_info.User_loc[i].longitude.degrees
    
        user_point = Point( d_user_lon, d_user_lat, 0 )
        sat_point  = Point( d_sat_lon,  d_sat_lat, 0 ) 
        geometry   = LineString([user_point, sat_point])
        
        State = None
        
        gdf_4326.loc[gdf_4326.shape[0]] = [
            SAT_name,
            SAT_ID,
            TIME,
            SAT_EL,
            SAT_AZ,
            SAT_lat,
            SAT_lon,
            SAT_h,
            geometry,
            State] 
    
    gdf_4326.set_geometry('geometry')
    gdf_4326.crs = {'init': 'epsg:4326'}
    
    return gdf_4326 